In [70]:
# ==================
#
# IMPORTS
#
# ==================
import pandas as pd
import numpy as np
import dill
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# ==================
#
# RAW DATA 
# Reference: E. A. Lopez-Rojas , A. Elmir, and S. Axelsson. 
# "PaySim: A financial mobile money simulator for fraud detection". 
# In: The 28th European Modeling and Simulation Symposium-EMSS, 
# Larnaca, Cyprus. 2016
#
# ==================
'''
raw_df = pd.read_csv('./data/paysim1/PS_20174392719_1491204439457_log.csv')
'''

"\nraw_df = pd.read_csv('./data/paysim1/PS_20174392719_1491204439457_log.csv')\n"

In [3]:
'''
dill.dump(obj=raw_df, file=open('./data/raw_df.pkl', mode='wb'))
'''

"\ndill.dump(obj=raw_df, file=open('raw_df.pkl', mode='wb'))\n"

In [4]:
raw_df = dill.load(file=open('./data/raw_df.pkl', mode='rb'))

In [5]:
# ==================
#
# INITIAL CHECKS
#
# ==================
df = raw_df.copy()

In [6]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
step              int64
type              object
amount            float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
# ==================
#
# CLEAN DATA
#
# ==================

In [8]:
# =============
# RENAME FEATURES
df = df.rename(columns={'step':'hour', 'nameOrig':'name_orig', 
                        'oldbalanceOrg':'init_bal_orig', 
                        'newbalanceOrig':'new_bal_orig', 
                        'nameDest':'name_dest',
                        'oldbalanceDest':'init_bal_dest',
                        'newbalanceDest':'new_bal_dest',
                        'isFraud':'is_fraud','isFlaggedFraud':'is_flagged'})

In [9]:
# ============
# CREATE CUSTOMER AND MERCHANT FEATURES
df['type_orig'] = df['name_orig'].map(lambda name: name[0])

In [10]:
df['type_orig'].unique()

array(['C'], dtype=object)

In [11]:
# Drop type_origin
df = df.drop(columns=['type_orig'])

In [12]:
df['type_dest'] = df['name_dest'].map(lambda name: name[0])

In [13]:
df['type_dest'].unique()

array(['M', 'C'], dtype=object)

In [15]:
# ---- CONVERT type_destination TO DUMMIES ----
df = pd.get_dummies(data=df, columns=['type_dest'])

In [16]:
df = df.drop(columns=['type_dest_M'])

In [17]:
df = df.rename(columns={'type_dest_C':'is_cust_dest'})

In [18]:
# ---- REARRANGE FEATURES ----
df = df[['hour', 'type', 'amount', 'name_orig', 'init_bal_orig', 'new_bal_orig',
         'name_dest', 'dest_is_cust', 'init_bal_dest', 'new_bal_dest', 
         'is_flagged', 'is_fraud']]

In [19]:
df.head()

,hour,type,amount,name_orig,init_bal_orig,new_bal_orig,name_dest,dest_is_cust,init_bal_dest,new_bal_dest,is_flagged,is_fraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,1,0.0,0.0,0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,1,21182.0,0.0,0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0,0.0,0.0,0,0


In [22]:
# ---- TYPE DUMMIES ----
df = pd.get_dummies(data = df, columns=['type'], drop_first=True)

In [23]:
df.head()

,hour,amount,name_orig,init_bal_orig,new_bal_orig,name_dest,dest_is_cust,init_bal_dest,new_bal_dest,is_flagged,is_fraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0,0.0,0.0,0,0,0,0,1,0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0,0.0,0.0,0,0,0,0,1,0
2,1,181.00,C1305486145,181.0,0.00,C553264065,1,0.0,0.0,0,1,0,0,0,1
3,1,181.00,C840083671,181.0,0.00,C38997010,1,21182.0,0.0,0,1,1,0,0,0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0,0.0,0.0,0,0,0,0,1,0


In [25]:
 df.columns

Index(['hour', 'amount', 'name_orig', 'init_bal_orig', 'new_bal_orig',
       'name_dest', 'dest_is_cust', 'init_bal_dest', 'new_bal_dest',
       'is_flagged', 'is_fraud', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER'],
      dtype='object')

In [26]:
df = df[['type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER', 
        'hour', 'amount', 'name_orig', 'init_bal_orig', 'new_bal_orig',
        'name_dest', 'dest_is_cust', 'init_bal_dest', 'new_bal_dest',
        'is_flagged', 'is_fraud']]

In [29]:
df = df.rename(columns={'type_CASH_OUT':'is_type_cash_out',
                  'type_DEBIT':'is_type_debit',
                  'type_PAYMENT':'is_type_payment',
                  'type_TRANSFER':'is_type_transfer'})

In [32]:
df.head()

,is_type_cash_out,is_type_debit,is_type_payment,is_type_transfer,hour,amount,name_orig,init_bal_orig,new_bal_orig,name_dest,is_cust_dest,init_bal_dest,new_bal_dest,is_flagged,is_fraud
0,0,0,1,0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0,0.0,0.0,0,0
1,0,0,1,0,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0,0.0,0.0,0,0
2,0,0,0,1,1,181.00,C1305486145,181.0,0.00,C553264065,1,0.0,0.0,0,1
3,1,0,0,0,1,181.00,C840083671,181.0,0.00,C38997010,1,21182.0,0.0,0,1
4,0,0,1,0,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0,0.0,0.0,0,0


In [47]:
# ==================
# NULL/BOGUS VALUES

# ---- NULL VALUES ----
df.isnull().sum().sum()

0

In [54]:
# ---- EMPTY STRING VALUES ----
(df=='').sum().sum()

0

In [53]:
(df==' ').sum().sum()

0

In [57]:
# ---- ZEROS ----
(df==0).sum()/len(df)*100

is_type_cash_out    64.833669
is_type_debit       99.348822
is_type_payment     66.185392
is_type_transfer    91.624378
hour                 0.000000
amount               0.000251
name_orig            0.000000
init_bal_orig       33.043762
new_bal_orig        56.730812
name_dest            0.000000
is_cust_dest        33.814608
init_bal_dest       42.504314
new_bal_dest        38.340071
is_flagged          99.999749
is_fraud            99.870918
dtype: float64

Notes about 0s:
* A surprisingly large 57% of the transactionss have an originator new balance of 0.
* A whopping 99.87% of the data is labeled is_fraud=0 - meaning only 0.13% of the data is actually fraud.
* From the dataset description we know that the merchant data is unknown; it is all 0s.

In [63]:
df[df['is_cust_dest']==0][['init_bal_dest', 'new_bal_dest']].sum()

init_bal_dest    0.0
new_bal_dest     0.0
dtype: float64

In [67]:
df['name_orig'].nunique() / len(df)

0.9985362947967975

In [68]:
df['name_dest'].nunique() / len(df)

0.4278680795018404

In [ ]:
sns.pairplot(data=df)